In [7]:
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from stellargraph import StellarGraph
from stellargraph.layer import GCN, LinkEmbedding
from stellargraph.mapper import FullBatchLinkGenerator
from tensorflow import keras

pop_size=100
benchmark='RosenBrock'
dimension = 10

In [8]:
def positive_and_negative_links(pos_edges,neg_edges):
    pos = pos_edges[["Src", "Dst"]].values.tolist()
    if not isinstance(neg_edges,list):
        neg_tuple =neg_edges[["Src", "Dst"]].values.tolist()
    else:
        neg_tuple =neg_edges
    neg = random.choices(neg_tuple,k=len(pos))
    return pos, neg

In [9]:
def get_labels(pos_test,neg_test,node_features):
    edge_ids_test =np.array(pos_test+neg_test)
    edge_labels_test = np.repeat([1, 0], [len(pos_test), len(neg_test)])
    G_test = StellarGraph(node_features, pd.DataFrame(list(pos_test),columns=["source","target"]), source_column="source", target_column="target", node_type_default="superior", edge_type_default="dominant")
    return G_test, edge_ids_test, edge_labels_test

In [10]:
def build_graph(generation):
    square_edges =pd.read_csv(f"ranker/generations/{generation}.csv")

    negetive_edges = square_edges[square_edges['Weight'] == 0]
    positive_edges = square_edges[square_edges['Weight'] == 1]

    # keep older edges in graph, and predict more recent edges
    edges_train, edges_test = train_test_split(positive_edges, test_size=0.25)

    features = pd.read_csv(f"ranker/features.csv")
    # square = StellarGraph(edges=square_edges)
    # square_named = StellarGraph(node_features=features,
    #     edges=pd.DataFrame(
    # {"source": src, "target":dst}), node_type_default="superior", edge_type_default="dominant"
    # )
    pos, neg = positive_and_negative_links(edges_train,negetive_edges)
    negetive_edges=negetive_edges[["Src", "Dst"]].values.tolist()
    negetive_edges = [i for i in negetive_edges if i not in neg]
    pos_test, neg_test = positive_and_negative_links(edges_test,negetive_edges)
    # edge_splitter_test = EdgeSplitter(square_first_second)
    G_test, edge_ids_test, edge_labels_test = get_labels(pos_test,neg_test,features)

    print(G_test.info())
    # edge_splitter_train = EdgeSplitter(G_test)
    G_train, edge_ids_train, edge_labels_train = get_labels(pos,neg,features)
    print(G_train.info())
    return G_train, edge_ids_train, edge_labels_train,G_test, edge_ids_test, edge_labels_test

In [11]:
def build_model(epochs,G_train,edge_ids_train,edge_labels_train,G_test,edge_ids_test, edge_labels_test):
    train_gen = FullBatchLinkGenerator(G_train, method="gcn")
    train_flow = train_gen.flow(edge_ids_train, edge_labels_train)
    test_gen = FullBatchLinkGenerator(G_test, method="gcn")
    test_flow = test_gen.flow(edge_ids_test, edge_labels_test)
    gcn = GCN(
        layer_sizes=[32,64, 128], activations=["relu", "relu","relu"], generator=train_gen, dropout=0.3
    )
    x_inp, x_out = gcn.in_out_tensors()
    prediction = LinkEmbedding(activation="relu", method="ip")(x_out)
    prediction = keras.layers.Reshape((-1,))(prediction)
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.01),
        loss=keras.metrics.binary_crossentropy,
        metrics=["acc"],
    )

    init_train_metrics = model.evaluate(train_flow)
    init_test_metrics = model.evaluate(test_flow)

    print("\nTrain Set Metrics of the initial (untrained) model:")
    for name, val in zip(model.metrics_names, init_train_metrics):
        print("\t{}: {:0.4f}".format(name, val))

    print("\nTest Set Metrics of the initial (untrained) model:")
    for name, val in zip(model.metrics_names, init_test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    history = model.fit(
        train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
    )
    train_metrics = model.evaluate(train_flow)
    test_metrics = model.evaluate(test_flow)

    print("\nTrain Set Metrics of the trained model:")
    for name, val in zip(model.metrics_names, train_metrics):
        print("\t{}: {:0.4f}".format(name, val))

    print("\nTest Set Metrics of the trained model:")
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))


In [12]:
G_train, edge_ids_train, edge_labels_train,G_test, edge_ids_test, edge_labels_test=build_graph(1)
build_model(100,G_train,edge_ids_train,edge_labels_train,G_test,edge_ids_test,edge_labels_test)

StellarGraph: Undirected multigraph
 Nodes: 3603, Edges: 500

 Node types:
  superior: [3603]
    Features: float32 vector, length 10
    Edge types: superior-dominant->superior

 Edge types:
    superior-dominant->superior: [500]
        Weights: all 1 (default)
        Features: none
StellarGraph: Undirected multigraph
 Nodes: 3603, Edges: 1498

 Node types:
  superior: [3603]
    Features: float32 vector, length 10
    Edge types: superior-dominant->superior

 Edge types:
    superior-dominant->superior: [1498]
        Weights: all 1 (default)
        Features: none
Using GCN (local pooling) filters...
Using GCN (local pooling) filters...


/Users/vidagharavian/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1/1 [==============================] - 0s 44ms/step - loss: 5.3529 - acc: 0.0000e+00

Train Set Metrics of the initial (untrained) model:
	loss: 0.7165
	acc: 0.0000

Test Set Metrics of the initial (untrained) model:
	loss: 5.3529
	acc: 0.0000
Epoch 1/100
1/1 - 1s - loss: 2.4974 - acc: 0.0000e+00 - val_loss: 1.1027 - val_acc: 0.0000e+00 - 923ms/epoch - 923ms/step
Epoch 2/100
1/1 - 0s - loss: 1.1255 - acc: 0.0000e+00 - val_loss: 1.3973 - val_acc: 0.0000e+00 - 61ms/epoch - 61ms/step
Epoch 3/100
1/1 - 0s - loss: 1.4014 - acc: 0.0000e+00 - val_loss: 0.9727 - val_acc: 0.0000e+00 - 64ms/epoch - 64ms/step
Epoch 4/100
1/1 - 0s - loss: 1.0920 - acc: 0.0000e+00 - val_loss: 0.6825 - val_acc: 0.0000e+00 - 70ms/epoch - 70ms/step
Epoch 5/100
1/1 - 0s - loss: 0.8318 - acc: 0.0000e+00 - val_loss: 2.0210 - val_acc: 0.0000e+00 - 72ms/epoch - 72ms/step
Epoch 6/100
1/1 - 0s - loss: 1.0252 - acc: 0.0000e+00 - val_loss: 0.8539 - val_acc: 0.0000e+00 - 63ms/epoch - 63ms/step
Epoch 7/100
1/1 - 0s - loss: 0.739

1/1 - 0s - loss: 0.6951 - acc: 0.0000e+00 - val_loss: 0.6856 - val_acc: 0.0000e+00 - 60ms/epoch - 60ms/step
Epoch 67/100
1/1 - 0s - loss: 0.6936 - acc: 0.0000e+00 - val_loss: 0.6850 - val_acc: 0.0000e+00 - 57ms/epoch - 57ms/step
Epoch 68/100
1/1 - 0s - loss: 0.7149 - acc: 0.0000e+00 - val_loss: 0.6915 - val_acc: 0.0000e+00 - 56ms/epoch - 56ms/step
Epoch 69/100
1/1 - 0s - loss: 0.6850 - acc: 0.0000e+00 - val_loss: 0.7002 - val_acc: 0.0000e+00 - 57ms/epoch - 57ms/step
Epoch 70/100
1/1 - 0s - loss: 0.6870 - acc: 0.0000e+00 - val_loss: 0.7137 - val_acc: 0.0000e+00 - 55ms/epoch - 55ms/step
Epoch 71/100
1/1 - 0s - loss: 0.6906 - acc: 0.0000e+00 - val_loss: 0.7367 - val_acc: 0.0000e+00 - 55ms/epoch - 55ms/step
Epoch 72/100
1/1 - 0s - loss: 0.6886 - acc: 0.0000e+00 - val_loss: 0.7547 - val_acc: 0.0000e+00 - 57ms/epoch - 57ms/step
Epoch 73/100
1/1 - 0s - loss: 0.6858 - acc: 0.0000e+00 - val_loss: 0.7762 - val_acc: 0.0000e+00 - 57ms/epoch - 57ms/step
Epoch 74/100
1/1 - 0s - loss: 0.6940 - acc: 0

In [15]:
def build_GraphSAGE(epochs,G_train,edge_ids_train,edge_labels_train,G_test,edge_ids_test, edge_labels_test):
    from stellargraph.mapper import GraphSAGELinkGenerator
    from stellargraph.layer import GraphSAGE, HinSAGE, link_classification
    batch_size = 20
    epochs = epochs
    num_samples = [20, 10]
    train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
    train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)
    test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
    test_flow = test_gen.flow(edge_ids_test, edge_labels_test)
    layer_sizes = [64, 128]
    graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
    )
    x_inp, x_out = graphsage.in_out_tensors()
    prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method="ip")(x_out)
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=["acc"],
    )
    init_train_metrics = model.evaluate(train_flow)
    init_test_metrics = model.evaluate(test_flow)

    print("\nTrain Set Metrics of the initial (untrained) model:")
    for name, val in zip(model.metrics_names, init_train_metrics):
        print("\t{}: {:0.4f}".format(name, val))

    print("\nTest Set Metrics of the initial (untrained) model:")
    for name, val in zip(model.metrics_names, init_test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)
    train_metrics = model.evaluate(train_flow)
    test_metrics = model.evaluate(test_flow)

    print("\nTrain Set Metrics of the trained model:")
    for name, val in zip(model.metrics_names, train_metrics):
        print("\t{}: {:0.4f}".format(name, val))

    print("\nTest Set Metrics of the trained model:")
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    return model

In [ ]:
build_GraphSAGE(20,G_train,edge_ids_train,edge_labels_train,G_test,edge_ids_test,edge_labels_test)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


/Users/vidagharavian/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


150/150 [==============================] - 4s 26ms/step - loss: 0.7729 - acc: 0.5060


In [14]:
for i in range(1,30):
    G_train, edge_ids_train, edge_labels_train,G_test, edge_ids_test, edge_labels_test=build_graph(i)
    build_GraphSAGE(20,G_train,edge_ids_train,edge_labels_train,G_test,edge_ids_test,edge_labels_test)

StellarGraph: Undirected multigraph
 Nodes: 3603, Edges: 500

 Node types:
  superior: [3603]
    Features: float32 vector, length 10
    Edge types: superior-dominant->superior

 Edge types:
    superior-dominant->superior: [500]
        Weights: all 1 (default)
        Features: none
StellarGraph: Undirected multigraph
 Nodes: 3603, Edges: 1498

 Node types:
  superior: [3603]
    Features: float32 vector, length 10
    Edge types: superior-dominant->superior

 Edge types:
    superior-dominant->superior: [1498]
        Weights: all 1 (default)
        Features: none
link_classification: using 'ip' method to combine node embeddings into edge embeddings


/Users/vidagharavian/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


50/50 [==============================] - 1s 25ms/step - loss: 0.7246 - acc: 0.5030

Train Set Metrics of the initial (untrained) model:
	loss: 0.7392
	acc: 0.4997

Test Set Metrics of the initial (untrained) model:
	loss: 0.7246
	acc: 0.5030
Epoch 1/20
150/150 - 7s - loss: 0.7231 - acc: 0.5180 - val_loss: 0.7482 - val_acc: 0.5110 - 7s/epoch - 44ms/step
Epoch 2/20
150/150 - 5s - loss: 0.7160 - acc: 0.5187 - val_loss: 0.7220 - val_acc: 0.5200 - 5s/epoch - 33ms/step
Epoch 3/20
150/150 - 5s - loss: 0.7129 - acc: 0.5130 - val_loss: 0.7223 - val_acc: 0.5330 - 5s/epoch - 32ms/step
Epoch 4/20
150/150 - 5s - loss: 0.7120 - acc: 0.5177 - val_loss: 0.7356 - val_acc: 0.5250 - 5s/epoch - 32ms/step
Epoch 5/20
150/150 - 5s - loss: 0.7092 - acc: 0.5230 - val_loss: 0.7224 - val_acc: 0.5210 - 5s/epoch - 32ms/step
Epoch 6/20
150/150 - 5s - loss: 0.7054 - acc: 0.5097 - val_loss: 0.7324 - val_acc: 0.5330 - 5s/epoch - 32ms/step
Epoch 7/20
150/150 - 5s - loss: 0.7015 - acc: 0.5334 - val_loss: 0.7374 - val_ac

/Users/vidagharavian/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


50/50 [==============================] - 1s 26ms/step - loss: 0.7464 - acc: 0.5131

Train Set Metrics of the initial (untrained) model:
	loss: 0.7843
	acc: 0.4920

Test Set Metrics of the initial (untrained) model:
	loss: 0.7464
	acc: 0.5131
Epoch 1/20
150/150 - 6s - loss: 0.7356 - acc: 0.4936 - val_loss: 0.7868 - val_acc: 0.5000 - 6s/epoch - 41ms/step
Epoch 2/20
150/150 - 5s - loss: 0.7216 - acc: 0.4977 - val_loss: 0.7858 - val_acc: 0.4960 - 5s/epoch - 32ms/step
Epoch 3/20
150/150 - 5s - loss: 0.7191 - acc: 0.5161 - val_loss: 0.7974 - val_acc: 0.5030 - 5s/epoch - 32ms/step
Epoch 4/20
150/150 - 5s - loss: 0.7097 - acc: 0.5264 - val_loss: 0.7643 - val_acc: 0.5100 - 5s/epoch - 32ms/step
Epoch 5/20
150/150 - 5s - loss: 0.7140 - acc: 0.5164 - val_loss: 0.7683 - val_acc: 0.5100 - 5s/epoch - 32ms/step
Epoch 6/20
150/150 - 5s - loss: 0.7005 - acc: 0.5315 - val_loss: 0.7416 - val_acc: 0.5241 - 5s/epoch - 32ms/step
Epoch 7/20
150/150 - 5s - loss: 0.7033 - acc: 0.5331 - val_loss: 0.7721 - val_ac

...

/Users/vidagharavian/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


51/51 [==============================] - 1s 27ms/step - loss: 1.4869 - acc: 0.5000

Train Set Metrics of the initial (untrained) model:
	loss: 1.5866
	acc: 0.5000

Test Set Metrics of the initial (untrained) model:
	loss: 1.4869
	acc: 0.5000
Epoch 1/20
151/151 - 6s - loss: 0.7449 - acc: 0.5060 - val_loss: 1.1570 - val_acc: 0.5000 - 6s/epoch - 42ms/step
Epoch 2/20
151/151 - 5s - loss: 0.7282 - acc: 0.4904 - val_loss: 1.0776 - val_acc: 0.5000 - 5s/epoch - 33ms/step
Epoch 3/20
151/151 - 5s - loss: 0.7239 - acc: 0.5013 - val_loss: 1.0333 - val_acc: 0.5000 - 5s/epoch - 33ms/step
Epoch 4/20
151/151 - 5s - loss: 0.7199 - acc: 0.5000 - val_loss: 0.9903 - val_acc: 0.5000 - 5s/epoch - 34ms/step
Epoch 5/20
151/151 - 5s - loss: 0.7214 - acc: 0.5063 - val_loss: 0.9761 - val_acc: 0.5000 - 5s/epoch - 34ms/step
Epoch 6/20
151/151 - 5s - loss: 0.7144 - acc: 0.4950 - val_loss: 0.9766 - val_acc: 0.5000 - 5s/epoch - 33ms/step
Epoch 7/20
151/151 - 5s - loss: 0.7195 - acc: 0.5013 - val_loss: 0.9891 - val_ac

/Users/vidagharavian/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


50/50 [==============================] - 2s 37ms/step - loss: 1.3021 - acc: 0.5000

Train Set Metrics of the initial (untrained) model:
	loss: 1.3987
	acc: 0.5000

Test Set Metrics of the initial (untrained) model:
	loss: 1.3021
	acc: 0.5000
Epoch 1/20
148/148 - 8s - loss: 0.7455 - acc: 0.4875 - val_loss: 1.0490 - val_acc: 0.5000 - 8s/epoch - 52ms/step
Epoch 2/20
148/148 - 6s - loss: 0.7329 - acc: 0.4861 - val_loss: 1.1301 - val_acc: 0.5000 - 6s/epoch - 38ms/step
Epoch 3/20
148/148 - 6s - loss: 0.7245 - acc: 0.5074 - val_loss: 1.0944 - val_acc: 0.5000 - 6s/epoch - 38ms/step
Epoch 4/20


KeyboardInterrupt: 